In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv('C:/Users/pulkit/SolarDataProcessed.csv',index_col=0)
print(df)

PRECTOT      QV2M     PS     TS    KT  ALLSKY_SFC_LW_DWN  \
0         0.00  0.003886  99.11  11.85  0.68               3.97   
1         0.00  0.003804  99.05  11.48  0.68               3.97   
2         0.00  0.003462  99.13  11.86  0.76               4.42   
3         0.00  0.003457  99.06  12.04  0.76               4.42   
4         0.00  0.002913  99.31  10.09  0.73               4.29   
...        ...       ...    ...    ...   ...                ...   
24587     1.77  0.020821  97.83  32.17  0.57               5.71   
24588     6.70  0.020814  97.68  30.82  0.54               5.40   
24589     4.94  0.020763  97.63  30.32  0.54               5.40   
24590     1.81  0.020122  97.45  32.49  0.53               5.35   
24591     1.11  0.019812  97.41  32.37  0.53               5.35   

       ALLSKY_SFC_SW_DWN  
0                   6.50  
1                   6.50  
2                   6.36  
3                   6.36  
4                   6.50  
...                  ...  
24587        

In [10]:
df['KT'] = df['KT'].astype(float)

In [5]:
df[df == -999] = np.nan
(df<0).any()

PRECTOT              False
QV2M                 False
PS                   False
TS                   False
KT                   False
ALLSKY_SFC_LW_DWN    False
ALLSKY_SFC_SW_DWN    False
dtype: bool

In [11]:
df.corr()

,PRECTOT,QV2M,PS,TS,KT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_SW_DWN
PRECTOT,1.000000,0.451316,-0.266209,0.112126,-0.458312,-0.232486,0.324511
QV2M,0.451316,1.000000,-0.679591,0.501087,-0.516028,0.015321,0.786639
PS,-0.266209,-0.679591,1.000000,-0.888651,0.337548,-0.443796,-0.873155
TS,0.112126,0.501087,-0.888651,1.000000,-0.204434,0.591189,0.842778
KT,-0.458312,-0.516028,0.337548,-0.204434,1.000000,0.603887,-0.505506
ALLSKY_SFC_LW_DWN,-0.232486,0.015321,-0.443796,0.591189,0.603887,1.000000,0.272502
ALLSKY_SFC_SW_DWN,0.324511,0.786639,-0.873155,0.842778,-0.505506,0.272502,1.000000


In [4]:
import numpy as np # linear algebra
from scipy import stats # statistics
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # plotting tools
import seaborn as sns # advanced plotting tools
sns.set(style="white")

def corrPairs(df):
    #Pairwise correlation matrix
    corr = df.corr() # Compute the correlation matrix
    mask = np.zeros_like(corr, dtype=np.bool) # make mask
    mask[np.triu_indices_from(mask)] = True # mask upper triangle
    sns.heatmap(corr, mask=mask, cmap='coolwarm', center=0, square=True, linewidths=.3, annot=True, cbar=False)

#df['WeekOfYear'] = df.index.date # add week to view correlation

plt.figure(figsize=(15,15))
corrPairs(df)

In [7]:
import re
df.replace(r'^\s*$', np.nan, regex=True)

In [8]:
df.fillna(df.mean(), inplace=True)

In [2]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
x = df[df.columns[:10]]
y = df['ALLSKY_SFC_SW_DWN']
x_train, x_test, y_train, y_test = train_test_split(x, y , train_size = 0.7, random_state =  90)
#Select numerical columns which needs to be normalized
train_norm = x_train[x_train.columns[0:10]]
test_norm = x_test[x_test.columns[0:10]]
# Normalize Training Data 
std_scale = preprocessing.StandardScaler().fit(train_norm)
x_train_norm = std_scale.transform(train_norm)
#Converting numpy array to dataframe
training_norm_col = pd.DataFrame(x_train_norm, index=train_norm.index, columns=train_norm.columns) 
x_train.update(training_norm_col)
print (x_train.head())
# Normalize Testing Data by using mean and SD of training set
x_test_norm = std_scale.transform(test_norm)
testing_norm_col = pd.DataFrame(x_test_norm, index=test_norm.index, columns=test_norm.columns) 
x_test.update(testing_norm_col)
print (x_train.head())

In [102]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.data import Dataset
import keras
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.optimizers import SGD
from keras import optimizers
from matplotlib import pyplot as plt
from keras import backend as K
#Select predictors
x = df[df.columns[0:6]]
#Target variable 
y = df['ALLSKY_SFC_SW_DWN']
#Split data into train and test 
x_train, x_test, y_train, y_test = train_test_split(x, y , train_size = 0.8, random_state =  0)


model = keras.Sequential([
 keras.layers.Dense(5,input_dim=6,activation='relu'),
 keras.layers.Dense(5,activation='relu'),  
 keras.layers.Dense(1,activation='softplus')
 ])

def r2_loss(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='mean_squared_error',
              metrics=['mse', 'mae', 'mape','accuracy', r2_score])
history = model.fit(X_train, y_train, batch_size = 500, validation_split = 0.20, epochs = 200)
y_pred = model.predict(x_test)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.semilogx(y_test, color = 'red', label = 'Real data')
plt.semilogx(y_pred, color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()

plt.plot(history.history['mean_squared_error'])
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['mean_absolute_percentage_error'])
plt.show()

In [ ]:
metrics=['accuracy'] 
in compile function
Regression
compile(optimizer='adadelta', loss='mse', metrics=['mse', 'accuracy'])
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
          callbacks=[TestCallback((X_test, Y_test))])
compile(optimizer='adadelta', loss='mse', metrics=['mse', 'accuracy'])
